In [1]:
from Process import PlateReader
import pandas as pd

### Step 1: Load Sample Information
* Ideally will be in the form of a dataframe. This dataframe should be standardized in the original documentation where we originally made the sample. 
* Either load locally or pull from the google team drive (still working on fixing pulling from Drive)

In [10]:
sample_info = pd.read_csv('05_07_21_CB_Info')
sample_info

,5CB concentration wtf,ethanol concentration wtf,water concentration wtf,5CB amount mass g,5CB amount volume uL,ethanol amount mass g,ethanol amount volume uL,water amount mass g,water amount volume uL,5CB-ethanol-stock amount volume uL,ethanol-stock amount volume uL,water-stock amount volume uL
0,0.00000,0.100000,0.900000,0.00000,0.000000,0.100000,126.694539,0.900000,900.000000,0.0,126.742712,900.000000
1,0.00025,0.100000,0.899750,0.00025,0.247525,0.100000,126.694539,0.899750,899.750000,100.0,26.990237,899.750000
2,0.00000,0.177778,0.822222,0.00000,0.000000,0.177778,225.234737,0.822222,822.222222,0.0,225.320377,822.222222
3,0.00025,0.177778,0.821972,0.00025,0.247525,0.177778,225.234737,0.821972,821.972222,100.0,125.567902,821.972222
4,0.00050,0.177778,0.821722,0.00050,0.495050,0.177778,225.234737,0.821722,821.722222,200.0,25.815427,821.722222
5,0.00000,0.255556,0.744444,0.00000,0.000000,0.255556,323.774934,0.744444,744.444444,0.0,323.898043,744.444444
6,0.00025,0.255556,0.744194,0.00025,0.247525,0.255556,323.774934,0.744194,744.194444,100.0,224.145567,744.194444
7,0.00050,0.255556,0.743944,0.00050,0.495050,0.255556,323.774934,0.743944,743.944444,200.0,124.393092,743.944444
8,0.00075,0.255556,0.743694,0.00075,0.742574,0.255556,323.774934,0.743694,743.694444,300.0,24.640617,743.694444
9,0.00000,0.333333,0.666667,0.00000,0.000000,0.333333,422.315132,0.666667,666.666667,0.0,422.475708,666.666667


### Step 2: Load UV-Vis Data
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
         
* Sometimes when using the plate reader the scanning protocol scans the whole plate versus only the wells of interest. This can be changed, however it is common to forget. If you do forget to do this, make sure to trim the dataframe to match the same length/order of wells as the sample info loaded in Step 1.

* ***If loading from single cuvette reader***: 
    * Still documenting.

***Step 2a:*** Provide the path of the excel sheet for the plate reader. With this you will also need to provide the **Sheet Name** of for correct excel sheet to be extracted. Since multiple plates are expected to be used at once, provide the Sheet names as a ***list of string names***. This will return a list of dataframes of each plate in the order the sheet names provided.

In [11]:
path = r"C:\Users\Edwin\Downloads\05_07_21_5CB.xlsx"
plate_names = ['Sheet1']
plate_dfs = PlateReader.extract_plates(path, plate_names)

***Step 2b:*** With the dataframes from ***Step2a***, given that all the plates read originally were done using the same protcol, then we can merge these dataframes into one. The only thing we need to edit is when originally formatting the excel sheets, since we copied the wavelength row in each sheet it only needs to be present once. A way to avoid to this is to copy it only in the first plate sheet you are using. 

Should be length of sample_info + 1 (will remove the wavelength column shortly).

In [12]:
merged_df = PlateReader.merge_wavelength_dfs(plate_dfs)
merged_df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
Wavelength,300.000,305.000,310.000,315.000,320.000,325.000,330.000,335.000,340.000,345.000,...,755.000,760.000,765.000,770.000,775.000,780.000,785.000,790.000,795.000,800.000
A1,0.278,0.194,0.163,0.144,0.124,0.110,0.098,0.085,0.074,0.068,...,0.046,0.046,0.046,0.046,0.046,0.046,0.046,0.046,0.045,0.045
A2,2.867,2.803,2.776,2.758,2.736,2.631,2.500,2.413,2.347,2.295,...,0.561,0.553,0.544,0.535,0.526,0.518,0.509,0.501,0.492,0.485
A3,0.312,0.232,0.202,0.183,0.163,0.148,0.136,0.122,0.110,0.104,...,0.057,0.057,0.057,0.057,0.056,0.056,0.056,0.055,0.055,0.055
A4,2.842,2.775,2.749,2.728,2.691,2.571,2.441,2.356,2.295,2.243,...,0.553,0.544,0.535,0.526,0.517,0.508,0.500,0.492,0.483,0.475
A5,3.057,2.984,2.954,2.930,2.917,2.893,2.766,2.667,2.602,2.551,...,1.205,1.189,1.173,1.156,1.140,1.124,1.107,1.092,1.076,1.061
A6,0.316,0.220,0.185,0.165,0.141,0.125,0.111,0.094,0.082,0.075,...,0.045,0.045,0.045,0.045,0.045,0.044,0.044,0.044,0.044,0.044
A7,2.696,2.616,2.579,2.532,2.429,2.276,2.160,2.085,2.029,1.984,...,0.655,0.647,0.638,0.629,0.620,0.612,0.603,0.595,0.586,0.578
A8,2.963,2.884,2.848,2.820,2.793,2.723,2.593,2.503,2.444,2.399,...,1.379,1.366,1.353,1.339,1.326,1.313,1.300,1.287,1.273,1.260
A9,3.051,2.982,2.950,2.926,2.908,2.877,2.752,2.659,2.596,2.550,...,1.538,1.524,1.510,1.496,1.480,1.466,1.452,1.438,1.422,1.408


***Step 2c:*** Sometimes when samples are extremely turbid or have some sort of material like foam or cream the plate reader absorbance is overflowed and returns as 'OVRFLW'. This will prevent you from handling the entire set of data, so what can be done is replace the 'OVRFLW' values with a holder number. It is reccomended you select a very high holder number so you can track and isolate it easier if needed. 

In [13]:
merged_df = PlateReader.detect_ovflw(merged_df, holder = 30)
merged_df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
Wavelength,300.000,305.000,310.000,315.000,320.000,325.000,330.000,335.000,340.000,345.000,...,755.000,760.000,765.000,770.000,775.000,780.000,785.000,790.000,795.000,800.000
A1,0.278,0.194,0.163,0.144,0.124,0.110,0.098,0.085,0.074,0.068,...,0.046,0.046,0.046,0.046,0.046,0.046,0.046,0.046,0.045,0.045
A2,2.867,2.803,2.776,2.758,2.736,2.631,2.500,2.413,2.347,2.295,...,0.561,0.553,0.544,0.535,0.526,0.518,0.509,0.501,0.492,0.485
A3,0.312,0.232,0.202,0.183,0.163,0.148,0.136,0.122,0.110,0.104,...,0.057,0.057,0.057,0.057,0.056,0.056,0.056,0.055,0.055,0.055
A4,2.842,2.775,2.749,2.728,2.691,2.571,2.441,2.356,2.295,2.243,...,0.553,0.544,0.535,0.526,0.517,0.508,0.500,0.492,0.483,0.475
A5,3.057,2.984,2.954,2.930,2.917,2.893,2.766,2.667,2.602,2.551,...,1.205,1.189,1.173,1.156,1.140,1.124,1.107,1.092,1.076,1.061
A6,0.316,0.220,0.185,0.165,0.141,0.125,0.111,0.094,0.082,0.075,...,0.045,0.045,0.045,0.045,0.045,0.044,0.044,0.044,0.044,0.044
A7,2.696,2.616,2.579,2.532,2.429,2.276,2.160,2.085,2.029,1.984,...,0.655,0.647,0.638,0.629,0.620,0.612,0.603,0.595,0.586,0.578
A8,2.963,2.884,2.848,2.820,2.793,2.723,2.593,2.503,2.444,2.399,...,1.379,1.366,1.353,1.339,1.326,1.313,1.300,1.287,1.273,1.260
A9,3.051,2.982,2.950,2.926,2.908,2.877,2.752,2.659,2.596,2.550,...,1.538,1.524,1.510,1.496,1.480,1.466,1.452,1.438,1.422,1.408


# Step 3: Merge with Sample Info Dataframe

The first step to merge to sample info dataframe is to format the plate reader dataframe to have the wavelengths as the column names versus as an another row. We can do this by isolating the Wavelength row, adding the appropiate unit the string of each element of the wavlength row and reassigning this new list as the column names. We finalize this by dropping the wavlength row.

In [14]:
platereader_df = PlateReader.rehead_wavelengths(merged_df)
# platereader_df.insert(0, 'Gain', [85,85,85,85, 95, 95, 95, 95])
platereader_df

,300.0nm,305.0nm,310.0nm,315.0nm,320.0nm,325.0nm,330.0nm,335.0nm,340.0nm,345.0nm,...,755.0nm,760.0nm,765.0nm,770.0nm,775.0nm,780.0nm,785.0nm,790.0nm,795.0nm,800.0nm
A1,0.278,0.194,0.163,0.144,0.124,0.110,0.098,0.085,0.074,0.068,...,0.046,0.046,0.046,0.046,0.046,0.046,0.046,0.046,0.045,0.045
A2,2.867,2.803,2.776,2.758,2.736,2.631,2.500,2.413,2.347,2.295,...,0.561,0.553,0.544,0.535,0.526,0.518,0.509,0.501,0.492,0.485
A3,0.312,0.232,0.202,0.183,0.163,0.148,0.136,0.122,0.110,0.104,...,0.057,0.057,0.057,0.057,0.056,0.056,0.056,0.055,0.055,0.055
A4,2.842,2.775,2.749,2.728,2.691,2.571,2.441,2.356,2.295,2.243,...,0.553,0.544,0.535,0.526,0.517,0.508,0.500,0.492,0.483,0.475
A5,3.057,2.984,2.954,2.930,2.917,2.893,2.766,2.667,2.602,2.551,...,1.205,1.189,1.173,1.156,1.140,1.124,1.107,1.092,1.076,1.061
A6,0.316,0.220,0.185,0.165,0.141,0.125,0.111,0.094,0.082,0.075,...,0.045,0.045,0.045,0.045,0.045,0.044,0.044,0.044,0.044,0.044
A7,2.696,2.616,2.579,2.532,2.429,2.276,2.160,2.085,2.029,1.984,...,0.655,0.647,0.638,0.629,0.620,0.612,0.603,0.595,0.586,0.578
A8,2.963,2.884,2.848,2.820,2.793,2.723,2.593,2.503,2.444,2.399,...,1.379,1.366,1.353,1.339,1.326,1.313,1.300,1.287,1.273,1.260
A9,3.051,2.982,2.950,2.926,2.908,2.877,2.752,2.659,2.596,2.550,...,1.538,1.524,1.510,1.496,1.480,1.466,1.452,1.438,1.422,1.408
A10,0.278,0.197,0.166,0.148,0.129,0.115,0.103,0.090,0.080,0.074,...,0.048,0.048,0.048,0.048,0.048,0.048,0.047,0.047,0.047,0.047


Next we reset both indexes of the two dataframes we are to merge and concatnate them together. We drop and reset indexes to ensure any past dependency is forgotten, so you must make sure your sample info dataframe and plate reader dataframe are ordered correctly where row 1...n of one dataframe referes to row 1...n of the other. 

In [16]:
sample_info.reset_index(drop=True, inplace=True)
platereader_df.reset_index(drop=True, inplace=True)
combined_df = pd.concat([sample_info, platereader_df], axis = 1) # this is called combine hotdog. 

,5CB concentration wtf,ethanol concentration wtf,water concentration wtf,5CB amount mass g,5CB amount volume uL,ethanol amount mass g,ethanol amount volume uL,water amount mass g,water amount volume uL,5CB-ethanol-stock amount volume uL,...,755.0nm,760.0nm,765.0nm,770.0nm,775.0nm,780.0nm,785.0nm,790.0nm,795.0nm,800.0nm
0,0.00000,0.100000,0.900000,0.00000,0.000000,0.100000,126.694539,0.900000,900.000000,0.0,...,0.046,0.046,0.046,0.046,0.046,0.046,0.046,0.046,0.045,0.045
1,0.00025,0.100000,0.899750,0.00025,0.247525,0.100000,126.694539,0.899750,899.750000,100.0,...,0.561,0.553,0.544,0.535,0.526,0.518,0.509,0.501,0.492,0.485
2,0.00000,0.177778,0.822222,0.00000,0.000000,0.177778,225.234737,0.822222,822.222222,0.0,...,0.057,0.057,0.057,0.057,0.056,0.056,0.056,0.055,0.055,0.055
3,0.00025,0.177778,0.821972,0.00025,0.247525,0.177778,225.234737,0.821972,821.972222,100.0,...,0.553,0.544,0.535,0.526,0.517,0.508,0.500,0.492,0.483,0.475
4,0.00050,0.177778,0.821722,0.00050,0.495050,0.177778,225.234737,0.821722,821.722222,200.0,...,1.205,1.189,1.173,1.156,1.140,1.124,1.107,1.092,1.076,1.061
5,0.00000,0.255556,0.744444,0.00000,0.000000,0.255556,323.774934,0.744444,744.444444,0.0,...,0.045,0.045,0.045,0.045,0.045,0.044,0.044,0.044,0.044,0.044
6,0.00025,0.255556,0.744194,0.00025,0.247525,0.255556,323.774934,0.744194,744.194444,100.0,...,0.655,0.647,0.638,0.629,0.620,0.612,0.603,0.595,0.586,0.578
7,0.00050,0.255556,0.743944,0.00050,0.495050,0.255556,323.774934,0.743944,743.944444,200.0,...,1.379,1.366,1.353,1.339,1.326,1.313,1.300,1.287,1.273,1.260
8,0.00075,0.255556,0.743694,0.00075,0.742574,0.255556,323.774934,0.743694,743.694444,300.0,...,1.538,1.524,1.510,1.496,1.480,1.466,1.452,1.438,1.422,1.408
9,0.00000,0.333333,0.666667,0.00000,0.000000,0.333333,422.315132,0.666667,666.666667,0.0,...,0.048,0.048,0.048,0.048,0.048,0.048,0.047,0.047,0.047,0.047


# Step 4: Format Dataframe as csv and Upload to Drive.
We simply call a method of the dataframe while ensuring to drop the index (again to remove and previous dependency). 

***If uploading from desktop***: Just grab and upload like any other file.

***If uploading from jupyter notebook***: Save the csv anywhere with a name that is not relevant, reccomended is something like "Temp File" as you will be uploading it to google drive under a different name. Then use the Gdrive module of functions from the Prepare package. Read the README in this package to understand how to use it, it will require a certain setup of credentials. 


In [17]:
temp_hold_path = r"05_07_21_5CB_Abs"
combined_df_csv = combined_df.to_csv(temp_hold_path, index=False)

,Total Volume,CdOl 84mM volpct,TOP 1mM volpct,ZnOl 84mM volpct,Time Sonicated,200nm,205nm,210nm,215nm,220nm,...,855nm,860nm,865nm,870nm,875nm,880nm,885nm,890nm,895nm,900nm
0,5.0,75.0,25.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,5.0,75.0,25.0,0.0,0.0,-0.007,-0.015,-0.007,-0.007,-0.006,...,-0.006,-0.006,-0.006,-0.006,-0.005,-0.006,-0.005,-0.005,-0.005,-0.005
2,5.0,75.0,24.5,0.5,0.0,0.047,0.042,0.052,0.051,0.045,...,-0.007,-0.007,-0.007,-0.007,-0.006,-0.006,-0.006,-0.005,-0.005,-0.005
3,5.0,75.0,24.5,0.5,0.0,0.036,0.036,0.038,0.042,0.040,...,-0.006,-0.006,-0.006,-0.006,-0.004,-0.004,-0.004,-0.003,-0.003,-0.003
4,5.0,75.0,24.0,1.0,0.0,0.027,0.028,0.032,0.038,0.031,...,-0.004,-0.004,-0.004,-0.004,-0.002,-0.003,-0.002,-0.002,-0.002,-0.001
5,5.0,75.0,24.0,1.0,0.0,0.038,0.044,0.046,0.049,0.048,...,-0.005,-0.005,-0.005,-0.005,-0.003,-0.004,-0.003,-0.003,-0.003,-0.002
6,5.0,75.0,20.0,5.0,0.0,0.042,0.044,0.045,0.050,0.048,...,-0.008,-0.008,-0.008,-0.008,-0.006,-0.007,-0.006,-0.006,-0.006,-0.006
7,5.0,75.0,20.0,5.0,0.0,0.046,0.051,0.055,0.055,0.052,...,-0.007,-0.007,-0.007,-0.007,-0.005,-0.006,-0.005,-0.005,-0.005,-0.004
8,5.0,75.0,25.0,0.0,10.0,0.144,0.134,0.148,0.159,0.152,...,-0.005,-0.005,-0.005,-0.005,-0.004,-0.004,-0.003,-0.003,-0.003,-0.003
9,5.0,75.0,25.0,0.0,10.0,0.111,0.101,0.116,0.121,0.116,...,-0.009,-0.008,-0.009,-0.009,-0.008,-0.009,-0.007,-0.007,-0.007,-0.007
